<a href="https://colab.research.google.com/github/davis9ja/im-srg_tensorflow/blob/master/testing_tensorflow_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import sys
from scipy.integrate import odeint, ode
import matplotlib.pyplot as plt
print(tf.__version__)

1.13.1


In [2]:
# --- BUILD HAMILTONIAN -----------
ref = np.asarray([1,1,1,1,0,0,0,0])
holes = np.asarray([0,1,2,3])
particles = np.asarray([4,5,6,7])

# one-body basis is just all possible single particle states
# B1 = tf.constant(holes+particles)#np.append(holes,particles)
B1 = np.append(holes,particles)

# we build two-body basis with all possible pairs of single particle states
B2 = []
ind2B = {}

count = 0
for h in holes:
    for h2 in holes:
        B2.append((h,h2))
        ind2B[(h,h2)] = count
        count += 1

for h in holes:
    for p in particles:
        B2.append((h,p))
        ind2B[(h,p)] = count
        count += 1
        
for p in particles:
    for h in holes:
        B2.append((p,h))
        ind2B[(p,h)] = count
        count += 1
            
for p in particles:
    for p2 in particles:
        B2.append((p,p2)) 
        ind2B[(p,p2)] = count
        count += 1           

# the one-body Hamiltonian is diagonal, with elements given by P-1 or 
# single particle state index divided by 2 (rounding down)
H1B = np.zeros((len(B1), len(B1)))
for i in range(len(B1)):
    H1B[i,i] = np.floor_divide(i,2)

# the two-body Hamiltonian is sparse; only non-zero contributions 
# where pq=rs or pq=sr=-rs
H2B = np.zeros((len(B2), len(B2)))
for i in range(len(B2)):
    for j in range(len(B2)):
        p, q = B2[i]
        r, s = B2[j]
        
        pp = np.floor_divide(p,2)
        qp = np.floor_divide(q,2)
        rp = np.floor_divide(r,2)
        sp = np.floor_divide(s,2)
        
        ps = 1 if p%2==0 else -1
        qs = 1 if q%2==0 else -1
        rs = 1 if r%2==0 else -1
        ss = 1 if s%2==0 else -1
        
        if pp != qp or rp != sp:
            continue
        if ps == qs or rs == ss:
            continue
        if ps == rs and qs == ss:
            H2B[i,j] = -0.25
        if ps == ss and qs == rs:
            H2B[i,j] = 0.25

B1_len = len(B1)
H2B_t = np.zeros((B1_len,B1_len,B1_len,B1_len))
for p in B1:
    for q in B1:
        for r in B1:
            for s in B1:
                H2B_t[p,q,r,s] = H2B[ind2B[(p,q)], ind2B[(r,s)]]
        
# test2 = np.einsum('ijij->ij',H2B_t)
# print(test2)
# for i in holes:
#   for j in holes:
#         print(H2B[ind2B[(i,j)], ind2B[(i,j)]])
#         print(H2B_t[i,j,i,j])
# print(np.shape(list(map(lambda i: H2B_t[i,:,i,:], holes))))
# test = list(map(lambda i,j: H2B_t[i,j,i,j],holes,particles))
# print(np.shape(test))

# covers na - nb
occA = np.zeros_like(H2B_t)
for a in B1:
    for b in B1:
        occA[a,b,a,b] = ref[a] - ref[b]
        
# covers (1-na-nb)
occB = np.zeros_like(H2B_t)
for a in B1:
    for b in B1:
        occB[a,b,a,b] = 1 - ref[a] - ref[b]
        
# covers na*nb + (1-na-nb)*nc
occC = np.zeros((B1_len,B1_len,B1_len,B1_len,B1_len,B1_len))
for a in B1:
    for b in B1:
        for c in B1:
            occC[a,b,c,a,b,c] = ref[a]*ref[b] + (1-ref[a]-ref[b])*ref[c]

# covers na*nb*(1-nc-nd) + na*nb*nc*nd
occD = np.zeros((B1_len,B1_len,B1_len,B1_len,B1_len,B1_len,B1_len,B1_len))
for a in B1:
    for b in B1:
        for c in B1:
            for d in B1:
                occD[a,b,c,d,a,b,c,d] = ref[a]*ref[b]*(1-ref[c]-ref[d])+ref[a]*ref[b]*ref[c]*ref[d]

ref_t = tf.convert_to_tensor(ref)

B1_t = tf.convert_to_tensor(B1, dtype=tf.int32)
holes = tf.convert_to_tensor(holes, dtype=tf.int32)
particles = tf.convert_to_tensor(particles, dtype=tf.int32)

H1B_t = tf.convert_to_tensor(H1B, dtype=tf.float32)
H2B_t = tf.convert_to_tensor(H2B_t, dtype=tf.float32)

occA_t = tf.convert_to_tensor(occA, dtype=tf.float32)
occB_t = tf.convert_to_tensor(occB, dtype=tf.float32)
occC_t = tf.convert_to_tensor(occC, dtype=tf.float32)
occD_t = tf.convert_to_tensor(occD, dtype=tf.float32)

In [3]:
# --- NORMAL ORDER HAMILTONIAN -----------
# Calculate 0b, 1b, 2b pieces 
#
# zero-body piece is scalar
# one-body piece is rank 2 tensor
# two-body piece is rank 4 tensor

# - Calculate 0b tensor
# E = tf.Variable(0.0)
contr_1b = tf.map_fn(lambda i: H1B_t[i,i], holes, dtype=tf.float32)
contr_2b = tf.map_fn(lambda i: H2B_t[i,:,i,:], holes, dtype=tf.float32)

E_1b = tf.reduce_sum(contr_1b, 0)
E_2b = 0.5*tf.reduce_sum(contr_2b, [0,1,2])
E = tf.add_n([E_1b, E_2b])

# - Calculate 1b tensor
# f = tf.Variable(tf.identity(H1B_t))
# f = tf.identity(H1B_t)
contr_2b = tf.map_fn(lambda i: H2B_t[:,i,:,i], holes, dtype=tf.float32)
contr_2b = tf.reduce_sum(contr_2b,0) # sum over holes

f = tf.add_n([H1B_t, contr_2b])

# - Calculate 2b tensor
G = tf.identity(H2B_t)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
# --- SET UP WHITE'S GENERATOR -----------

# TODO: try reduce_sum, einsum, tensordot?, other methods

def white(f, G, holes, particles):
  
    # - Calculate 1b generator tensor
    eta1B = tf.identity(f)
    eta1Bph = eta1B[particles[0]:particles[-1]+1,holes[0]:holes[-1]+1]
    fhh = f[holes[0]:holes[-1]+1,holes[0]:holes[-1]+1]
    fpp = f[particles[0]:particles[-1]+1,particles[0]:particles[-1]+1]
    Gphph = G[particles[0]:particles[-1]+1, holes[0]:holes[-1]+1, particles[0]:particles[-1]+1, holes[0]:holes[-1]+1]
    Gphph = tf.reduce_sum(Gphph,[0,1]) # reduce to same rank as f
    
    Deltaph = tf.subtract(fpp, tf.add(fhh,Gphph))
    temp = tf.div_no_nan(eta1Bph,Deltaph)
    
    top_block = tf.zeros((4,4))
    bottom_block = tf.zeros((4,8))
    conc_top = tf.concat([top_block, temp],1)
    eta1B = tf.concat([conc_top, bottom_block], 0)
    
    # - Calculate 2b generator tensor
    eta2B = tf.zeros(G.shape)
#     return temp
    return eta1B
#     return (eta1B, eta2B)

In [5]:
# --- SET UP WEGNER'S GENERATOR -----------

def wegner(f, G):
# basis = tf.concat([holes, particles],0)

    # --- Need to decouple diagonal and off-diagonal elements; procedure in Ch.10 AACCNP

    # Decoupling 1B piece
    # indices are constructed by all possible combinations of particle-hole(hole-particle) states
    col_indices =tf.reshape(tf.concat([tf.broadcast_to(particles,[4,4]),tf.broadcast_to(holes,[4,4])],0),[-1])
    row_indices = tf.reshape(tf.transpose(tf.concat([tf.broadcast_to(holes,[4,4]),tf.broadcast_to(particles,[4,4])],1)),[-1])
    ph_indices = tf.stack([row_indices, col_indices], axis=1)
    ph_updates = tf.gather_nd(f, ph_indices)

    fod = tf.scatter_nd(ph_indices,ph_updates,f.shape)
    fd = tf.subtract(f,fod)

    # Decoupling 2B piece
    # indices are constructed by all possible combinations of pphh(hhpp) states
    ind1_C = tf.concat([tf.broadcast_to(holes,[64,4]), tf.broadcast_to(particles,[64,4])],1)
    ind1_TC = tf.transpose(ind1_C) 
    ind1 = tf.reshape(ind1_TC,[-1])

    ind2_C = tf.concat([tf.broadcast_to(holes,[16,16]),tf.broadcast_to(particles,[16,16])],1)
    ind2_TC = tf.transpose(ind2_C)
    ind2 = tf.reshape(ind2_TC,[-1])

    ind3_C = tf.concat([tf.broadcast_to(particles,[4,64]),tf.broadcast_to(holes,[4,64])],1)
    ind3_TC = tf.transpose(ind3_C) 
    ind3 = tf.reshape(ind3_TC,[-1])

    ind4_C = tf.concat([tf.broadcast_to(particles,[1,256]),tf.broadcast_to(holes,[1,256])],1)
    ind4_TC = tf.transpose(ind4_C)
    ind4 = tf.reshape(ind4_TC,[-1])

    pphh_indices = tf.stack([ind1,ind2,ind3,ind4],axis=1)
    pphh_updates = tf.gather_nd(G, pphh_indices)

    God = tf.scatter_nd(pphh_indices,pphh_updates,G.shape)
    Gd = tf.subtract(G,God)


    # --- 1B piece
#     eta1B = tf.Variable(tf.zeros(f.shape))

    # Calculate 1B-1B contribution
    fd_fod = tf.tensordot(fd,fod,1)
    fd_fod_T = tf.transpose(fd_fod)
    eta1B_1b1b = tf.subtract(fd_fod, fd_fod_T)

    # fod_fd = tf.tensordot(fod,fd,1)
    # eta1B = tf.assign_add(eta1B, tf.subtract(fd_fod, fod_fd))


    # Calculate 1B-2B contribution
    fd_God = tf.tensordot(fd, tf.tensordot(occA_t,God,([0,1],[2,0])),([0,1],[2,0]))
    fod_Gd = tf.tensordot(fod, tf.tensordot(occA_t,Gd,([0,1],[2,0])),([0,1],[2,0]))
    eta1B_1b2b = tf.subtract(fd_God, fod_Gd)


    # Calculate 2B-2B contribution
    Gd_God = tf.tensordot(Gd, tf.tensordot(occC_t,God,([0,1,2],[0,1,2])),([2,3,1],[0,1,2]))
    Gd_God_T = tf.transpose(Gd_God)
    scaled_sub = tf.scalar_mul(tf.constant(0.5),tf.subtract(Gd_God,Gd_God_T))
    eta1B_2b2b = scaled_sub
    
    eta1B = tf.add_n([eta1B_1b1b, eta1B_1b2b, eta1B_2b2b])

    #     God_Gd = tf.tensordot(God, tf.tensordot(occC_t,Gd,([0,1,2],[0,1,2])),([0,1,2],[2,3,1]))
    #     scaled_sub = tf.scalar_mul(tf.constant(0.5),tf.subtract(Gd_God,God_Gd))
    #     eta1B = tf.assign_add(eta1B, scaled_sub)




    # --- 2B piece
#     eta2B = tf.Variable(tf.zeros(G.shape))

    # Calculate 1B-2B contribution
    fdGod_fodGd_ij = tf.subtract( tf.tensordot(fd,God,[[1],[0]]), tf.tensordot(fod,Gd,[[1],[0]]) )
    fdGod_fodGd_ij_T = tf.transpose(fdGod_fodGd_ij, perm=[1,0,2,3])
    ij_term = tf.subtract(fdGod_fodGd_ij,fdGod_fodGd_ij_T)

    # Godfd_Gdfod_ij = tf.subtract( tf.tensordot(God,fd,([0],[1])), tf.tensordot(Gd,fod,([0],[1])) )
    # ij_term = tf.subtract(fdGod_fodGd_ij,Godfd_Gdfod_ij)

    fdGod_fodGd_kl = tf.subtract( tf.tensordot(fd,God,[[0],[2]]), tf.tensordot(fod,Gd,[[0],[2]]) )
    fdGod_fodGd_kl = tf.transpose(fdGod_fodGd_kl,perm=[1,2,0,3]) # permute back to i,j,k,l order
    fdGod_fodGd_kl_T = tf.transpose(fdGod_fodGd_kl,perm=[0,1,3,2])
    kl_term = tf.subtract(fdGod_fodGd_kl,fdGod_fodGd_kl_T)

    # Godfd_Gdfod_kl = tf.subtract( tf.tensordot(God,fd,([2],[0])), tf.tensordot(Gd,fod,([2],[0])) )
    # kl_term = tf.subtract(fdGod_fodGd_kl,Godfd_Gdfod_kl)

    eta2B_1b2b = tf.subtract(ij_term,kl_term)


    # Calculate 2B-2B contribution
    # GdGod_GodGd = tf.subtract( tf.tensordot(Gd,God,[[2,3],[0,1]]), tf.tensordot(God,Gd,[[2,3],[0,1]]) )
    # GdGod_GodGd_occB = tf.tensordot(occB_t,GdGod_GodGd,[[2,3],[0,1]])

    GdGod_occB = tf.tensordot(Gd, tf.tensordot(occB_t, God, [[0,1],[0,1]]), [[2,3],[0,1]])
    GodGd_occB = tf.tensordot(God, tf.tensordot(occB_t, Gd, [[0,1],[0,1]]), [[2,3],[0,1]])
    scaled_sub = tf.scalar_mul(tf.constant(0.5),tf.subtract(GdGod_occB,GodGd_occB))
    # GdGod_occB = tf.tensordot( Gd, tf.tensordot(occB_t,God,[[0,1],[0,1]]), [[2,3],[0,1]] )
    # GodGd_occB = tf.tensordot( God, tf.tensordot(occB_t,Gd,[[0,1],[0,1]]), [[2,3],[0,1]] )
    # scaled_sub = tf.scalar_mul(tf.constant(0.5), tf.subtract(GdGod_occB,GodGd_occB))
    eta2B_2b2b_B = scaled_sub

    GdGod = tf.tensordot(Gd,God,[[0,2],[2,0]])
    GdGod = tf.transpose(GdGod,perm=[0,2,1,3]) # permute back to i,j,k,l order
    GdGod_occA = tf.tensordot(occA_t,GdGod,[[2,3],[0,1]])
    # GdGod_occA = tf.tensordot( Gd, tf.tensordot(occA_t,God,[[0,1],[2,0]]), [[0,2],[0,1]] )
    # GdGod_occA = tf.transpose(GdGod_occA, perm=[0,2,1,3]) # permute back to i,j,k,l order
    GdGod_occA_Tij = tf.transpose(GdGod_occA,perm=[1,0,2,3])
    GdGod_occA_Tkl = tf.transpose(GdGod_occA,perm=[0,1,3,2])
    GdGod_occA_Tijkl = tf.transpose(GdGod_occA,perm=[1,0,3,2])
    sub1 = tf.subtract(GdGod_occA,GdGod_occA_Tij)
    sub2 = tf.subtract(sub1,GdGod_occA_Tkl)
    add3 = tf.add(sub2,GdGod_occA_Tijkl)

    eta2B_2b2b_A = add3
    
    eta2B = tf.add_n([eta2B_1b2b, eta2B_2b2b_B, eta2B_2b2b_A])
    
    return (eta1B, eta2B)

In [21]:
# --- WRITE FLOW EQUATIONS -----------

def flow(f, G, eta1B, eta2B):
#     eta1B, eta2B = wegner(f,G)

    # --- 0B piece
    # dE = tf.Variable(0.0)

    # Calculate 1B-1B contribution (full contraction)
    occA_e1 = tf.tensordot(occA_t, eta1B, [[0,1],[0,1]])
    occA_e1_f = tf.tensordot(occA_e1, f, [[0,1],[1,0]])
    dE_1b1b = tf.identity(occA_e1_f)

    # Calculate 2B-2B contribution (full contraction)
    e2_occD = tf.tensordot(eta2B, occD_t, [[0,1,2,3],[0,1,2,3]])
    e2_occD_G = 0.5*tf.tensordot(e2_occD, G, [[0,1,2,3],[2,3,0,1]])

    # e2_occD_G = tf.scalar_mul(tf.constant(0.5),e2_occD_G)
    dE_2b2b = tf.identity(e2_occD_G)

    dE = tf.add_n([dE_1b1b, dE_2b2b])

    # --- 1B piece
    # df = tf.Variable(tf.zeros(f.shape))

    # Calculate 1B-1B contribution (contraction over 1 index)
    e1_f = tf.tensordot(eta1B,f,[[1],[0]])
    e1_f_T = tf.transpose(e1_f)
    e1_f_add = tf.add(e1_f,e1_f_T)
    df_1b1b = tf.identity(e1_f_add)

    # Calculate 1B-2B contribution (contraction over 2 indices)
    occA_e1_G = tf.tensordot(occA_t, tf.tensordot(eta1B,G,[[0,1],[2,0]]), [[2,3],[0,1]])
    occA_f_e2 = tf.tensordot(occA_t, tf.tensordot(f,eta2B,[[0,1],[2,0]]), [[2,3],[0,1]])
    sub_1b2b = tf.subtract(occA_e1_G, occA_f_e2)
    df_1b2b = tf.identity(sub_1b2b)

    # Calculate 2B-2B contribution (contraction over 3 indices)
    e2_occC_G = tf.tensordot(eta2B, tf.tensordot(occC_t,G,[[3,4,5],[0,1,2]]), [[2,3,0],[0,1,2]])
    e2_occC_G_T = tf.transpose(e2_occC_G)
    add_2b2b = 0.5*tf.add(e2_occC_G,e2_occC_G_T)
    df_2b2b = tf.identity(add_2b2b)

    df = tf.add_n([df_1b1b, df_1b2b, df_2b2b])

    # --- 2B piece
    # dG = tf.Variable(tf.zeros(G.shape))

    # Calculate 1B-2B contribution (contraction over 1 index)
    e1G_fe2_ij = tf.subtract(tf.tensordot(eta1B,G,[[1],[0]]), tf.tensordot(f,eta2B,[[1],[0]]))
    e1G_fe2_ij_T = tf.transpose(e1G_fe2_ij, perm=[1,0,2,3])
    ij_term = tf.subtract(e1G_fe2_ij,e1G_fe2_ij_T)

    e1G_fe2_kl = tf.subtract(tf.tensordot(eta1B,G,[[0],[2]]), tf.tensordot(f,eta2B,[[0],[2]]))
    e1G_fe2_kl = tf.transpose(e1G_fe2_kl, perm=[1,2,0,3]) # permute to i,j,k,l order
    e1G_fe2_kl_T = tf.transpose(e1G_fe2_kl, perm=[0,1,3,2])
    kl_term = tf.subtract(e1G_fe2_kl,e1G_fe2_kl_T)

    dG_1b2b = tf.identity(tf.subtract(ij_term, kl_term))

    # Calculate 2B-2B contribution (occB term)
    e2_occB_G = tf.tensordot(eta2B, tf.tensordot(occB_t, G, [[2,3],[0,1]]), [[2,3],[0,1]])
    G_occB_e2 = tf.tensordot(G, tf.tensordot(occB_t, eta2B, [[2,3],[0,1]]), [[2,3],[0,1]])
    sub_term = 0.5*tf.subtract(e2_occB_G, G_occB_e2)
#     e2G_Ge2 = tf.subtract(tf.tensordot(eta2B,G,[[2,3],[0,1]]), tf.tensordot(G,eta2B,[[2,3],[0,1]]))
#     e2G_Ge2_occB = 0.5*tf.tensordot(occB_t,e2G_Ge2,[[2,3],[0,1]])

    dG_2b2b_B = tf.identity(sub_term)

    # Calculate 2B-2B contribution (occA term)
    e2G = tf.tensordot(eta2B, G, [[0,2],[2,0]])
    e2G = tf.transpose(e2G, perm=[0,2,1,3]) # permute back to i,j,k,l order
    e2G_occA = tf.tensordot(occA_t, e2G, [[2,3],[0,1]])
    e2G_occA_Tij = tf.transpose(e2G_occA, perm=[1,0,2,3])
    e2G_occA_Tkl = tf.transpose(e2G_occA, perm=[0,1,3,2])
    e2G_occA_Tijkl = tf.transpose(e2G_occA, perm=[1,0,3,2])
    sub1 = tf.subtract(e2G_occA, e2G_occA_Tij)
    sub2 = tf.subtract(sub1, e2G_occA_Tkl)
    add3 = tf.add(sub2, e2G_occA_Tijkl)

    dG_2b2b_A = tf.identity(add3)

    dG = tf.add_n([dG_1b2b, dG_2b2b_B, dG_2b2b_A])

    return (dE, df, dG)

In [22]:
# --- DEFINE DERIVATIVE TO PASS INTO ODEINT SOLVER -----------
# The input is an *evaluated graph* (numpy array); the output 
# is also an evaluated graph. Formatting in this way allows
# for compability with scipy.ode; the tensorflow ode 
# package will be deprecated in TF v2.0

def derivative(t, y):
#     print(y.shape)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    
    E, f, G = ravel(y)
    
    eta1B, eta2B = wegner(f, G)
    
    dE, df, dG = flow(f, G, eta1B, eta2B)
    
    dy = unravel(dE, df, dG)
    
#     print("executed")
    
    return dy.eval()

In [23]:
def unravel(E, f, G):
    unravel_E = tf.reshape(E, [-1])
    unravel_f = tf.reshape(f, [-1])
    unravel_G = tf.reshape(G, [-1])
    
    return tf.concat([unravel_E, unravel_f, unravel_G], 0)

def ravel(y):
    
    ravel_E = tf.reshape(y[0], ())
    ravel_f = tf.reshape(y[1:65], (8,8))
    ravel_G = tf.reshape(y[65:65+4096], (8,8,8,8))
    
    return(ravel_E, ravel_f, ravel_G)
    

In [26]:
# --- CAUTION: DO NOT EVALUATE PAST/FUTURE GRAPHS IN A SINGLE SESSION WHEN THOSE GRAPHS ARE DEPENDENT ---
# e.g., evaluating dE also evaluates eta2B,eta1B; if you try to evaluate eta2B,eta1B again, you will 
#       receive an unexpected result
# 
# --- The workaround is to use the var_print() method to "print" the value of the tensor you want;
#     I think the issue has something to do with tf.Variable and the assign_add() method. In the 
#     future, I would like to fix the issue. For now, though, I just want a running code.
#
# ************ ABOVE ISSUE HAS BEEN FIXED; WILL CLOSE OUT WARNING IN RELEASE VERSION *****************

# def var_print(session, tensor):
#     session.run(tf.global_variables_initializer())
#     print(session.run(tensor))



# with tf.Session() as sess:
#     y0 = unravel(E, f, G)
#     y0_eval = y0.eval()
     
#     t = 1
#     dy = derivative(t, y0_eval)
    
#     dE, df, dG = ravel(dy)
#     print(G[0,1,4,5].eval())
#     print(dG[0,1,4,5].eval())

with tf.Session() as sess:

    y0 = unravel(E, f, G)
    y0_eval = y0.eval()
#     print(y0_eval.shape)
    solver = ode(derivative,jac=None)
    solver.set_integrator('vode', method='bdf', order=5, nsteps=1000)
#     solver.set_f_params(len(f), len(Y))
    solver.set_initial_value(y0_eval, 0.)

    sfinal = 7
    ds = 0.1
    s_vals = []
    E_vals = []
    while solver.successful() and solver.t < sfinal:
        ys = solver.integrate(sfinal, step=True)
        Es, fs, Gs = ravel(ys) # outputs tensors
        print("scale param: {:0.4f} \t E = {:0.5f}".format(solver.t,Es.eval()))
        s_vals.append(solver.t)
        E_vals.append(Es)
        
        
    plt.plot(s_vals, E_vals)
    plt.show()


scale param: 0.0000 	 E = 1.50000
scale param: 0.0000 	 E = 1.50000
scale param: 0.0000 	 E = 1.50000
scale param: 0.0000 	 E = 1.50000
scale param: 0.0000 	 E = 1.50000
scale param: 0.0000 	 E = 1.50000
scale param: 0.0000 	 E = 1.49999
scale param: 0.0000 	 E = 1.49999
scale param: 0.0000 	 E = 1.49998
scale param: 0.0000 	 E = 1.49997
scale param: 0.0000 	 E = 1.49993
scale param: 0.0001 	 E = 1.49983
scale param: 0.0002 	 E = 1.49973
scale param: 0.0002 	 E = 1.49957
scale param: 0.0003 	 E = 1.49941
scale param: 0.0004 	 E = 1.49925
scale param: 0.0006 	 E = 1.49894
scale param: 0.0010 	 E = 1.49821
scale param: 0.0015 	 E = 1.49749
scale param: 0.0021 	 E = 1.49639


KeyboardInterrupt: 

In [ ]:
#     print(ravel)
#     print(dG[0,1,5,4].eval())

#     test = np.zeros((8,8,8,8))
#     test1 = np.zeros((8,8,8,8))
#     test2 = np.zeros_like(test)
#     test3 = np.zeros_like(test)
#     test4 = np.zeros_like(test)
#     eta2B_l = eta2B.eval()
#     for i in B1:
#         for j in B1:
#             for k in B1:
#                 for l in B1:
# #                     for a in B1:
#                     if(eta2B_l[i,j,k,l]>0):
#                         print(eta2B_l[i,j,k,l],i,j,k,l)
# #                         if(fd_l[a,k]*God_l[i,j,a,l] < 0 or fd_l[a,k]*God_l[i,j,a,l] > 0):
# #                             print(fd_l[a,k]*God_l[i,j,a,l], a,"|",i,j,k,l)
#                         test1[i,j,k,l] += fd_l[i,a]*God_l[a,j,k,l]
#                         test2[i,j,k,l] += fd_l[j,a]*God_l[a,i,k,l]
#                         test3[i,j,k,l] += fd_l[a,k]*God_l[i,j,a,l]
#                         test4[i,j,k,l] += fd_l[a,l]*God_l[i,j,a,k]
                        
#                         test[i,j,k,l] += fd_l[i,a]*God_l[a,j,k,l] \
#                                        - fd_l[j,a]*God_l[a,i,k,l] \
#                                        - fd_l[a,k]*God_l[i,j,a,l] \
#                                        + fd_l[a,l]*God_l[i,j,a,k]
#     print(test[0,1,4,5])
#     print(test1[0,1,4,5],test2[0,1,4,5],test3[0,1,4,5],test4[0,1,4,5])
# #     print(tf.transpose(test,perm=[0,1,3,2])[0,1,4,5].eval())
    
#     test_con1 = tf.tensordot(fd,God,[[1],[0]])
#     test_con2 = tf.transpose(test_con1,perm=[1,0,2,3])
#     test_con3 = tf.transpose(tf.tensordot(fd,God,[[0],[2]]),perm=[1,2,0,3])
#     test_con4 = tf.transpose(test_con3,perm=[0,1,3,2])
#     test_con = tf.add(tf.subtract(tf.subtract(test_con1,test_con2),test_con3),test_con4)
#     print(test_con1[0,1,4,5].eval(), test_con2[0,1,4,5].eval(),test_con3[0,1,4,5].eval(),test_con4[0,1,4,5].eval())
#     print(test_con[0,1,4,5].eval())
#     print("true,",test3[0,1,4,5])
#     print("test,",test_con3[0,1,4,5].eval())
#     print("tf,",tf.tensordot(fd,God,[[0],[2]])[4,0,1,5].eval())
#     print("tf_p,", tf.transpose(tf.tensordot(fd,God,[[0],[2]]),perm=[1,2,0,3])[0,1,4,5].eval())
#     print("numpy,",np.tensordot(fd_l,God_l,axes=((0),(2)))[0,1,4,5])
#     print(tf.transpose(fdGod_fodGd_kl,perm=[1,2,0,3])[0,1,4,5].eval())

# A = tf.constant([[1,2,3,4,5,6,7,8],
#                  [3,4,5,6,7,8,9,1], 
#                  [7,8,9,1,2,3,4,5], 
#                  [6,7,8,9,1,2,3,4],
#                  [1,2,3,4,5,6,7,8],
#                  [3,4,5,6,7,8,9,1], 
#                  [7,8,9,1,2,3,4,5], 
#                  [6,7,8,9,1,2,3,4] ])

# # indices = tf.constant([4,5,6,7, 4,5,6,7, 4,5,6,7, 4,5,6,7, 0,1,2,3, 0,1,2,3, 0,1,2,3, 0,1,2,3])
# indices =tf.reshape(tf.concat([tf.broadcast_to(particles,[4,4]),tf.broadcast_to(holes,[4,4])],0),[-1])

# # prepare row indices
# # row_indices = tf.range(tf.shape(indices)[0])
# # row_indices = tf.constant([0,0,0,0, 1,1,1,1, 2,2,2,2, 3,3,3,3, 4,4,4,4, 5,5,5,5, 6,6,6,6, 7,7,7,7])
# row_indices = tf.reshape(tf.transpose(tf.concat([tf.broadcast_to(holes,[4,4]),tf.broadcast_to(particles,[4,4])],1)),[-1])
# # row_indices = tf.reshape(tf)
# # zip row indices with column indices
# full_indices = tf.stack([row_indices, indices], axis=1)

# # retrieve values by indices
# # indices = tf.transpose(full_indices)
# # updates = tf.reshape(tf.gather_nd(A, full_indices),[2,2])
# updates = tf.gather_nd(A, full_indices)
# S = tf.scatter_nd(full_indices,updates,[8,8])

# test = tf.scatter_nd([[[1,0,0,1],[1,0,0,0]]],[[1,2]],[8,8,8,8])
# idx2b = tf.expand_dims(tf.stack([row_indices,row_indices,indices,indices],axis=1),0)
# test2b = tf.zeros([8,8,8,8])
# with tf.Session() as sess:
# #     print(full_indices.eval())
#     print(test[1,0,0,1].eval())
#     print(test[1,0,0,0].eval())
# #     print(tf.gather_nd([[[1,0,0,1],[1,0,0,0]]],test))
#     print(A.eval())
#     print("\n")
#     print(S.eval())
#     # print(full_indices.eval())
#     # print(tf.transpose(full_indices))
#     print(idx2b.eval())
#     print(tf.gather_nd(test2b,idx2b).eval())
# #     print(tf.concat([tf.broadcast_to(particles,[4,4]),tf.broadcast_to(holes,[4,4])],0).eval())
# #     print(tf.reshape(tf.concat([tf.broadcast_to(particles,[4,4]),tf.broadcast_to(holes,[4,4])],0),[-1]).eval())
# #     print(tf.reshape(tf.transpose(tf.broadcast_to(B1_t,[4,8])),[-1]).eval())
# #     print(tf.broadcast_to(tf.concat([holes,particles],1),[4,4]).eval())
# #     print(tf.reshape(test,[-1]).eval())
# # session.close()

In [ ]:
# a = tf.constant(1)
# shape = a.shape
# print(len(shape))

# def off_diagonal(tensor):
#     # returns off diagonal elements defined by indices ph,hp (1B) or pphh,hhpp (2B)
#     shape = tensor.shape
#     if(shape==() or len(shape)==1):
#         return 0
    
    
    

In [ ]:
# hols = np.asarray([0,1,2,3])
# pars = np.asarray([4,5,6,7])

# count = 0
# for p in pars:
#     for pp in pars:
#         for h in hols:
#             for hh in hols:
# #                 print(p,pp,h,hh)
#                 count += 1
                
# print(count)

# indices are p,pp,h,hh
# for every possible combination:
#
# p   --> switches every 64 states - 2^6
# pp  --> switches every 16 states - 2^4
# h   --> switches every 04 states - 2^2
# hh  --> swithces every 01 states - 2^0

# ind1_C = tf.concat([tf.broadcast_to(holes,[64,4]), tf.broadcast_to(particles,[64,4])],1)
# ind1_TC = tf.transpose(ind1_C) 
# ind1 = tf.reshape(ind1_TC,[-1])

# ind2_C = tf.concat([tf.broadcast_to(holes,[16,16]),tf.broadcast_to(particles,[16,16])],1)
# ind2_TC = tf.transpose(ind2_C)
# ind2 = tf.reshape(ind2_TC,[-1])

# ind3_C = tf.concat([tf.broadcast_to(particles,[4,64]),tf.broadcast_to(holes,[4,64])],1)
# ind3_TC = tf.transpose(ind3_C) 
# ind3 = tf.reshape(ind3_TC,[-1])

# ind4_C = tf.concat([tf.broadcast_to(particles,[1,256]),tf.broadcast_to(holes,[1,256])],1)
# ind4_TC = tf.transpose(ind4_C)
# ind4 = tf.reshape(ind4_TC,[-1])

# pphh_ind = tf.stack([ind1,ind2,ind3,ind4],axis=1)

# with tf.Session() as sess:
#     print(pphh_ind.eval())